In [ ]:
!pip install hashlib
!pip install time
!pip install json


In [24]:
!pip install blockchain

In [35]:
import hashlib as hash
import datetime as date
import blockchain
import blockchain
from cryptography.fernet import Fernet
from openpyxl import Workbook, load_workbook
import json

##LOGIN Y REGISTRO

En esta sección se establece el login y registro de los usuarios que dentro del sistema se registran y son acreditados bajo las macrocredenciales.

##Funciones

In [73]:
blockchain = Blockchain()

In [74]:
# Generar clave de encriptación al inicio del programa
clave = Fernet.generate_key()
cipher_suite = Fernet(clave)
ARCHIVO_ESTUDIANTES = "estudiantes.xlsx"  # Nombre del archivo Excel

def encriptar_datos(datos):
    return cipher_suite.encrypt(datos.encode())

def desencriptar_datos(datos_encriptados):
    return cipher_suite.decrypt(datos_encriptados).decode()

In [75]:
def crear_excel():
    try:
        wb = Workbook()
        ws = wb.active
        ws.append(["ID", "Nombre", "Contraseña encriptada"])

        estudiantes = [
            {"id": "001", "nombre": "Estudiante1", "contraseña": "password1"},
            {"id": "002", "nombre": "Estudiante2", "contraseña": "password2"}
        ]

        for estudiante in estudiantes:
            estudiante["contraseña"] = encriptar_datos(estudiante["contraseña"])
            ws.append([estudiante["id"], estudiante["nombre"], estudiante["contraseña"]])

        wb.save(ARCHIVO_ESTUDIANTES)
        print(f"Archivo '{ARCHIVO_ESTUDIANTES}' creado con éxito.")

    except Exception as e:
        print(f"Error al crear el archivo: {e}")

In [76]:
def login_estudiante(id_estudiante, contraseña_ingresada):
    try:
        wb = load_workbook(ARCHIVO_ESTUDIANTES)
        ws = wb.active

        for row in ws.iter_rows(min_row=2):
            if row[0].value == id_estudiante:
                contraseña_encriptada = row[2].value
                contraseña_desencriptada = desencriptar_datos(contraseña_encriptada)
                return contraseña_desencriptada == contraseña_ingresada

        return False  # Si no se encuentra el estudiante

    except FileNotFoundError:
        print(f"El archivo '{ARCHIVO_ESTUDIANTES}' no existe.")
        return False
    except Exception as e:
        print(f"Error durante el proceso de inicio de sesión: {e}")
        return False

In [77]:
def verificar_estudiante_existente(ws, id_estudiante, nombre):
    for row in ws.iter_rows(min_row=2):
        if row[0].value == id_estudiante or row[1].value == nombre:
            return True  # El estudiante ya está registrado
    return False

In [78]:
def registrar_estudiante(nombre, contraseña):
    id_estudiante = obtener_nuevo_id()

    try:
        with load_workbook(ARCHIVO_ESTUDIANTES) as wb:
            ws = wb.active
    except FileNotFoundError:
        wb = Workbook()
        ws = wb.active
        ws.append(["ID", "Nombre", "Contraseña encriptada"])

    if verificar_estudiante_existente(ws, id_estudiante, nombre):
        print("El estudiante ya está registrado.")
        return False

    if len(contraseña) == 0:
        print("La contraseña no puede estar vacía.")
        return False

    contraseña_encriptada = encriptar_datos(contraseña)
    ws.append([id_estudiante, nombre, contraseña_encriptada])

    try:
        wb.save(ARCHIVO_ESTUDIANTES)
        print(f"Estudiante registrado exitosamente. ID: {id_estudiante}")
        return True
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")
        return False



In [79]:
def obtener_lista_estudiantes():
  with load_workbook(ARCHIVO_ESTUDIANTES) as wb:
    ws = wb.active
    estudiantes = []
    for row in ws.iter_rows(min_row=2):
      estudiantes.append({
          "id": row[0].value,
          "nombre": row[1].value,
          "contraseña": row[2].value})
      return estudiantes


In [80]:
import hashlib
import datetime

class Macrocredencial:
    def __init__(self, emisor, receptor, contenido):
        self.emisor = emisor
        self.receptor = receptor
        self.contenido = contenido
        self.timestamp = datetime.datetime.utcnow()  # Utilizando tiempo UTC
        self.hash = self.calcular_hash()

    def calcular_hash(self):
        contenido_str = f"{self.emisor}{self.receptor}{self.contenido}{self.timestamp}"
        return hashlib.sha256(contenido_str.encode()).hexdigest()

    def __str__(self):
        return f"Emisor: {self.emisor}, Receptor: {self.receptor}, Contenido: {self.contenido}, Timestamp: {self.timestamp}, Hash: {self.hash}"

# Ejemplo de uso:
credencial = Macrocredencial("Alice", "Bob", "Mensaje secreto")
print(credencial)


Emisor: Alice, Receptor: Bob, Contenido: Mensaje secreto, Timestamp: 2023-11-30 13:05:32.744565, Hash: 2d010739cd9e59090af83096100e5935da5dc760d8014d3d7827f88e3c66b0cd


In [81]:
class Block:
    def __init__(self, macrocredencial, previous_hash):
        self.macrocredencial = macrocredencial
        self.previous_hash = previous_hash
        self.timestamp = datetime.datetime.utcnow().replace(microsecond=0).isoformat()  # Utilizando tiempo UTC y formato ISO 8601
        self.hash = self.calcular_hash()

    def calcular_hash(self):
        if not self.es_macrocredencial_valida():
            raise ValueError("Macrocredencial no válida")

        block_content = {
            "macrocredencial_hash": self.macrocredencial.hash,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp
        }
        block_content_serialized = json.dumps(block_content, sort_keys=True)
        return hashlib.sha256(block_content_serialized.encode()).hexdigest()

    def es_macrocredencial_valida(self):
        return hasattr(self.macrocredencial, 'hash') and hasattr(self.macrocredencial, 'timestamp')


In [82]:
class Blockchain:
    def __init__(self):
        self.chain = [self.genesis_block()]

    def genesis_block(self):
        genesis_macrocredencial = Macrocredencial("Universidad", "Estudiante", "Macrocredencial inicial")
        return Block(genesis_macrocredencial, "0")

    def agregar_bloque(self, new_block):
        new_block.previous_hash = self.chain[-1].hash
        new_block.hash = new_block.calcular_hash()
        self.chain.append(new_block)

    def imprimir_blockchain(self):
        for i, block in enumerate(self.chain):
            print(f"Block {i}")
            print(f"Timestamp: {block.timestamp}")
            print(f"Hash anterior: {block.previous_hash}")
            print(f"Hash: {block.hash}")
            if hasattr(block, 'macrocredencial'):
                print(f"Macrocredencial Emisor: {block.macrocredencial.emisor}")
                print(f"Macrocredencial Receptor: {block.macrocredencial.receptor}")
                print(f"Macrocredencial Contenido: {block.macrocredencial.contenido}")
            print()

def obtener_macrocredenciales_estudiante(id_estudiante, registro_estudiantes, blockchain):
    macrocredenciales_estudiante = []

    for block in blockchain.chain:
        if hasattr(block, 'macrocredencial') and block.macrocredencial.receptor == id_estudiante:
            macrocredenciales_estudiante.append(block.macrocredencial)

    # Verificar la integridad de las macrocredenciales en el registro de estudiantes
    macrocredenciales_validas = []
    for macrocredencial in macrocredenciales_estudiante:
        for estudiante in registro_estudiantes:
            if estudiante["id"] == id_estudiante:
                if macrocredencial.hash == estudiante["macrocredencial_hash"]:
                    macrocredenciales_validas.append(macrocredencial)

    return macrocredenciales_validas

In [83]:
def iniciar_sesion_y_obtener_macrocredenciales(id_estudiante, contraseña_ingresada, registro_estudiantes, blockchain):
    if login_estudiante(id_estudiante, contraseña_ingresada):
        print("¡Inicio de sesión exitoso!")

        # Obtener las macrocredenciales del estudiante después del inicio de sesión
        macrocredenciales_estudiante = obtener_macrocredenciales_estudiante(id_estudiante, registro_estudiantes, blockchain)

        if macrocredenciales_estudiante:
            print(f"Macrocredenciales del Estudiante {id_estudiante}:")
            for credencial in macrocredenciales_estudiante:
                print(f"Emisor: {credencial.emisor}, Contenido: {credencial.contenido}")
        else:
            print("No se encontraron macrocredenciales para este estudiante.")
    else:
        print("Inicio de sesión fallido. Verifica tus credenciales.")

In [84]:
def login_estudiante(id_estudiante, contraseña_ingresada):
    try:
        wb = load_workbook("/content/drive/MyDrive/Big data/estudiantes.xlsx")
        ws = wb.active

        for row in ws.iter_rows(min_row=2):
            if row[0].value == id_estudiante:
                contraseña_encriptada = row[2].value
                contraseña_desencriptada = desencriptar_datos(contraseña_encriptada)
                if contraseña_desencriptada == contraseña_ingresada:
                    return True  # Inicio de sesión exitoso
                else:
                    return False  # Contraseña incorrecta

        return False  # Si no se encuentra el estudiante

    except FileNotFoundError:
        print("El archivo de estudiantes no existe.")
        return False
    except Exception as e:
        print(f"Error durante el inicio de sesión: {e}")
        return False


In [85]:
def login_estudiante(id_estudiante, contraseña_ingresada):
    try:
        wb = load_workbook("/content/drive/MyDrive/Big data/estudiantes.xlsx")
        ws = wb.active

        for row in ws.iter_rows(min_row=2):
            if row[0].value == id_estudiante:
                contraseña_encriptada = row[2].value
                contraseña_desencriptada = desencriptar_datos(contraseña_encriptada)
                if contraseña_desencriptada == contraseña_ingresada:
                    return True  # Inicio de sesión exitoso
                else:
                    return False  # Contraseña incorrecta

        return False  # Si no se encuentra el estudiante

    except FileNotFoundError:
        print("El archivo de estudiantes no existe.")
        return False
    except Exception as e:
        print(f"Error durante el inicio de sesión: {e}")
        return False

##Ejecución

In [86]:
import hashlib
import datetime as date

# Suponiendo que las clases Macrocredencial, Block y Blockchain están definidas y se tiene una instancia válida de 'blockchain'

# Creación de nuevas macrocredenciales y bloques
macrocredencial_estudiante1 = Macrocredencial("Universidad A", "001", "Macrocredencial de Matemáticas")
block1 = Block(macrocredencial_estudiante1, blockchain.chain[-1].hash)
blockchain.agregar_bloque(block1)

macrocredencial_estudiante2 = Macrocredencial("Universidad B", "002", "Macrocredencial de Historia")
block2 = Block(macrocredencial_estudiante2, blockchain.chain[-1].hash)
blockchain.agregar_bloque(block2)

# Suponiendo que 'registro_estudiantes' es una lista de diccionarios con información de estudiantes y 'blockchain' es una instancia de la clase Blockchain

id_ingresado = "003"
contraseña_ingresada = "password3"

iniciar_sesion_y_obtener_macrocredenciales(id_ingresado, contraseña_ingresada, registrar_estudiante, blockchain)


# Suponiendo que 'registro_estudiantes' es una lista de diccionarios con información de estudiantes y 'blockchain' es una instancia de la clase Blockchain
macrocredencial_estudiante3 = Macrocredencial("Universidad A", "002", "Macrocredencial economía")
block3 = Block(macrocredencial_estudiante3, blockchain.chain[-1].hash)
blockchain.agregar_bloque(block3)

# Impresión de la cadena de bloques actualizada
print("Cadena de bloques actualizada:")
blockchain.imprimir_blockchain()


Inicio de sesión fallido. Verifica tus credenciales.
Cadena de bloques actualizada:
Block 0
Timestamp: 2023-11-30T13:05:32
Hash anterior: 0
Hash: a895ca6e01d404f5c81ade9e15e6770cf6df1e86cb8461ae9bae13892ce556e1
Macrocredencial Emisor: Universidad
Macrocredencial Receptor: Estudiante
Macrocredencial Contenido: Macrocredencial inicial

Block 1
Timestamp: 2023-11-30T13:05:39
Hash anterior: a895ca6e01d404f5c81ade9e15e6770cf6df1e86cb8461ae9bae13892ce556e1
Hash: c81df8d9306070956db0b62e63661ff007c9590232142b55ba6f36db08889402
Macrocredencial Emisor: Universidad A
Macrocredencial Receptor: 001
Macrocredencial Contenido: Macrocredencial de Matemáticas

Block 2
Timestamp: 2023-11-30T13:05:39
Hash anterior: c81df8d9306070956db0b62e63661ff007c9590232142b55ba6f36db08889402
Hash: c51b26a4e0150703644eab11d7bc8db45959a8dc0bf7ba04d38800a24c71aba0
Macrocredencial Emisor: Universidad B
Macrocredencial Receptor: 002
Macrocredencial Contenido: Macrocredencial de Historia

Block 3
Timestamp: 2023-11-30T13:

##Validación de la cadena

In [87]:
import hashlib
import datetime as date

# Definición de la clase Blockchain
class Blockchain:

    def __init__(self):
        self.chain = []

    def add_block(self, block):
        self.chain.append(block)

    def verify_chain(self):
        """
        Verifica la integridad de la cadena de bloques.

        Devuelve True si la cadena es válida, False si no lo es.
        """
        # Verificamos el bloque génesis
        if self.chain[0].hash != self.chain[0].calculate_hash():
            return False

        # Verificamos los bloques restantes
        for i in range(1, len(self.chain)):
            prev_block = self.chain[i - 1]
            curr_block = self.chain[i]

            # Verificamos el hash del bloque actual
            if curr_block.hash != curr_block.calculate_hash():
                return False

            # Verificamos que el hash previo del bloque actual coincida con el hash del bloque anterior
            if curr_block.previous_hash != prev_block.hash:
                return False

        return True

# Creación de una nueva cadena de bloques
blockchain = Blockchain()

# Creación de nuevos bloques
macrocredencial_estudiante1 = Macrocredencial("Universidad A", "001", "Macrocredencial de Matemáticas")
block1 = Block(macrocredencial_estudiante1, blockchain.chain[-1].hash)
blockchain.add_block(block1)

macrocredencial_estudiante2 = Macrocredencial("Universidad B", "002", "Macrocredencial de Historia")
block2 = Block(macrocredencial_estudiante2, blockchain.chain[-1].hash)
blockchain.add_block(block2)

# Impresión de la cadena de bloques
print("Cadena de bloques:")
print(blockchain.chain)

# Verificación de la integridad de la cadena de bloques
is_chain_valid = blockchain.verify_chain()
print(f"¿La cadena es válida? {is_chain_valid}")


def consensus(new_block, nodes):
    """
    Realiza el consenso para agregar un nuevo bloque a la cadena de bloques.

    Devuelve True si el bloque es aceptado, False si no lo es.
    """
    agreeing_nodes = 0

    for node in nodes:
        # Verificamos si el bloque propuesto se conecta correctamente al último bloque del nodo
        if new_block.previous_hash == node.blockchain[-1].hash and new_block.hash == new_block.calculate_hash():
            agreeing_nodes += 1

    # Si más del 50% de los nodos están de acuerdo, agregamos el bloque a todos los nodos
    if agreeing_nodes > len(nodes) / 2:
        for node in nodes:
            node.add_block(new_block)
        return True
    return False

# Nodo 1 propone un nuevo bloque
new_block = add_block(blockchain.chain[-1], "Dato desde Nodo 1")

# Verificamos si el consenso permite agregar el bloque
block_accepted = consensus(new_block, nodes)

# Verificamos si el bloque fue aceptado y la validez de los blockchains locales en todos los nodos
block_accepted, [node.verify_chain() for node in nodes]


def resolve_conflicts(nodes):
    """
    Resolve un conflicto entre cadenas de bloques.

    Actualiza todos los nodos para que tengan la cadena de bloques más larga.
    """
    # Buscamos la cadena de bloques más larga entre todos los nodos
    max_length = max([len(node.blockchain) for node in nodes])



IndexError: ignored

In [89]:
# Creación de nuevos bloques
if len(blockchain.chain) > 0:
    macrocredencial_estudiante1 = Macrocredencial("Universidad A", "001", "Macrocredencial de Matemáticas")
    block1 = Block(macrocredencial_estudiante1, blockchain.chain[-1].hash)
    blockchain.add_block(block1)
else:
    print("La cadena de bloques está vacía")

La cadena de bloques está vacía
